In [ ]:
pip install pyvis

In [3]:
import requests
import glob
import pandas as pd
from pyvis import network as net
import networkx as nx
import json
import os
from itertools import chain

In [4]:
def get_id(nick):
  METHOD_NAME="users.get"
  PARAMETERS=f"user_ids={nick}"
  ACCESS_TOKEN="4f0cdbfe4f0cdbfe4f0cdbfe404c19004544f0c4f0cdbfe2a1394c707dcdfeafd64a24f"
  V=5.154
  link=f'https://api.vk.com/method/{METHOD_NAME}?{PARAMETERS}&access_token={ACCESS_TOKEN}&v={V}'
  response = requests.get(link)
  try:
    id=response.json()['response'][0]['id']
  except IndexError:
    return 0
  return id

In [5]:
def get_friends(id):
  METHOD_NAME="friends.get"
  PARAMETERS=f"user_id={id}&fields=screen_name"
  ACCESS_TOKEN="4f0cdbfe4f0cdbfe4f0cdbfe404c19004544f0c4f0cdbfe2a1394c707dcdfeafd64a24f"
  V=5.154
  link=f'https://api.vk.com/method/{METHOD_NAME}?{PARAMETERS}&access_token={ACCESS_TOKEN}&v={V}'
  response = requests.get(link)
  output=[]
  try:
    resp_data=response.json()['response']['items']
  except KeyError:
    return 0
  for i in resp_data:
    output.append(i['id'])
  return output

In [6]:
path='/content/friends'
filenames=glob.glob(path+'/*.csv')
temp_data=[]
for fname in filenames:
  temp=pd.read_csv(fname)
  rows,cols=temp.shape
  for i in range(cols):
    vars=temp.iloc[:,i].values.tolist()
    vars=list(set(vars))
    if ' ' not in str(vars[0]):
      temp_data.append(pd.DataFrame(vars))
data=pd.concat(temp_data, ignore_index=True)
for i, value in data.iterrows():
  temp=value[0]
  if type(temp) is str:
    if 'https://vk.com/' in temp:
      temp=temp.rsplit('https://vk.com/')[1]
    id=get_id(temp)
    if id!=0:
      data.at[i,0]=id
    else:
      data.drop(labels=[i],inplace=True)
data.drop_duplicates(inplace=True,ignore_index=True)
data=data.rename(columns={0:'IDs'})
data

,IDs
0,209834587
1,308412461
2,45745684
3,232210943
4,275549140
5,188959578
6,513083713
7,606396724
8,163067034
9,54705450


In [8]:
friend_connect={}
new_id=[]
for _,value in data.iterrows():
  temp=get_friends(value[0])
  if temp==0:
    continue
  friend_connect[value[0]]=temp
  new_id+=temp
new_id=list(set(new_id))
for id in new_id:
  temp=get_friends(id)
  if temp==0:
    continue
  friend_connect[id]=temp
# os.remove(path+'/friends_connect.json')
with open(path+'/friends_connect.json', 'w', encoding='utf-8') as fh:
    fh.write(json.dumps(friend_connect, ensure_ascii=False))

In [9]:
with open(path+'/friends_connect.json', 'r', encoding='utf-8') as fh:
    friend_connect = json.load(fh)
g=net.Network(height="1000", width="100%", bgcolor="#222222", font_color="white", select_menu=True,notebook=True)#notebook=True
g.barnes_hut()
nodes = (list(friend_connect.keys()) + list(friend_connect.values()))
nodes = list(set(chain(*nodes)))
counter = 0
cc = len(nodes)
for i in nodes:
    g.add_node(i)
    counter += 1
    print(f'{counter * 100 / cc:.1f}', "% НОДЫ")
print("Ура")
counter = 0
cc = len(friend_connect)
for i in friend_connect:
    for j in friend_connect[i]:
        g.add_edge(i, j)
    counter+=1
    print(f'{counter*100/cc:.1f}',"% ЕДЖИ")
g.save_graph('friends.html')


KeyboardInterrupt: ignored